## installation and set up

In [ ]:
!pip install openai

In [ ]:
# setup

from google.colab import userdata
OPENAI_API_KEY=userdata.get('GROUP1_OPENAI_API_KEY')

from google.colab import drive
drive.mount('/content/drive')


from openai import OpenAI

client = OpenAI(
    api_key = OPENAI_API_KEY
)

### define assistants

for this tutorial we are going to assume that you have created your assistants in the openai assistant playground (you can also test them out there). Once you've done that, add them to the list below and choose the one(s) you'd like to use.

In [ ]:
# add your assistants here (or just hard code a single assistant as the one you want to use)
from google.colab import userdata

# Retrieve assistant IDs from user data
neanderthal_id = userdata.get('neanderthal_assistant_id')
proto_indo_european_id = userdata.get('proto_indo_european_assistant_id')
rising_star_id = userdata.get('rising_star_assistant_id')
sanxingdui_id = userdata.get('sanxingdui_assistant_id')

group1_assistants = {
    "neanderthal": neanderthal_id,
    "proto_indo_european": proto_indo_european_id,
    "rising_star": rising_star_id,
    "sanxingdui": sanxingdui_id
}

assistant_id = group1_assistants["sanxingdui"]

## update assistant with functions

In [ ]:
# function schema sample
my_function = {
  "type": "function",
  "function": {
    "name": "get_themes",
    "description": "Gets an array of AT LEAST 3 themes from the list of themes in response to any question asking for a story, folk tale, myth, or related narrative. These 3 themes should be the best for the type of story that the user has asked for.",
     "parameters": {
      "type": "object",
      "properties": {
        "themes": {
          "type": "array",
          "description": "List containing at between 3 and 7 themes selected from the specified array of themes",
          "minItems": 3,
          "items": {
            "type": "string",
            "enum": [
              "Adventure",
              "Courage",
              "Friendship",
              "Betrayal",
              "Love",
              "Mystery",
              "Quest",
              "Revenge",
              "Sacrifice",
              "Survival",
              "Tragedy",
              "Victory"
            ]
          }
        }
      },
      "required": ["themes"]
    }
  }
}


In [ ]:
# update assistant with functions

import datetime
from openai import OpenAI

timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# from previous step in this colab
new_assisant_name = f"demo-bot-w-functions-{timestamp}"


my_updated_assistant = client.beta.assistants.update(
  assistant_id,
  name=new_assisant_name,
  tools=[{"type": "retrieval"}, my_function],
  model="gpt-4-turbo-preview"
)

print(my_updated_assistant)

## thread cycle

### create

In [ ]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="can you create a myth about solar eclipses, where they came from, etc? Please do so by referencing any works you can retrieve (and tell me which works you referenced)"
)

print(thread)


### create the "run" which includes a message and the final prompt (PROMPT A)

### run the thread

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)

In [ ]:
import json

print(json.dumps(run.dict(), indent=4))

### check on the thread

and wait for completion (could take a minute or so).

In [ ]:
import time
import json

def wait_for_completion(client, thread_id, run_id):
    while True:
        run_info = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run_info.status}")

        if run_info.status == "completed":
            print("Run is complete.")
            break
        elif run_info.status == "requires_action":
            print(f"Run requires action.")
            break

        else:
            # Wait for a short period before checking again if the run is in any other state.
            print(f"Not done yet, status is {run_info.status}")
            time.sleep(5)  # Adjust sleep time as needed.

    return run_info

run_info = wait_for_completion(client, thread.id, run.id)

print(json.dumps(run_info.dict(), indent=4))

### retrieve the output and handle tool calls


plus new logic that sorts through ALL messages and only returns the last-produced JSON string.

In [ ]:
arguments_to_handle = run_info.required_action.submit_tool_outputs.tool_calls[0].function.arguments
tool_call_id = run_info.required_action.submit_tool_outputs.tool_calls[0].id

print(arguments_to_handle)
print(tool_call_id)

In [ ]:
# ask a different assistant to do something with this

import textwrap

def text_to_text(prompt):
    system_prompt = "You are a helpful assistant with a wealth of knowledge about folklore and myth from a variety of cultures across the globe."
    user_prompt = prompt
    response = client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    wrapped_text = textwrap.fill(response.choices[0].message.content, width=500)
    print(wrapped_text)
    return response.choices[0].message.content

tool_call_processed_content = text_to_text(f"can you help explain what Joseph Campbell would say about these themes? {json.dumps(arguments_to_handle)}")

In [ ]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
      {
        "tool_call_id": tool_call_id,
        "output": tool_call_processed_content
      }
    ]
)

### retrieve again until done

In [ ]:
import time
import json

def wait_for_completion(client, thread_id, run_id):
    while True:
        run_info = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run_info.status}")

        if run_info.status == "completed":
            print("Run is complete.")
            break
        elif run_info.status == "requires_action":
            print(f"Run requires action.")
            break

        else:
            # Wait for a short period before checking again if the run is in any other state.
            print(f"Not done yet, status is {run_info.status}")
            time.sleep(5)  # Adjust sleep time as needed.

    return run_info

run_info = wait_for_completion(client, thread.id, run.id)

print(json.dumps(run_info.dict(), indent=4))

In [ ]:
import json
import textwrap

messages = client.beta.threads.messages.list(thread_id=thread.id)

# FOR ALL MESSAGES UNCOMMENT THIS

# for message in messages.data:
#     message_info = client.beta.threads.messages.retrieve(
#         thread_id=thread.id,
#         message_id=message.id
#     )
#     print(json.dumps(message_info.dict(), indent=4))
#     message_text = message_info.content[0].text.value
#     print(textwrap.fill(message_text, width=50))

# JUST MOST RECENT MESSAGE

message_info = client.beta.threads.messages.retrieve(
    thread_id=thread.id,
    message_id=messages.data[0].id
)

print(json.dumps(message_info.dict(), indent=4))
message_text = message_info.content[0].text.value
print(textwrap.fill(message_text, width=50))

### optional = handle citations

In [ ]:
def process_citations(client, message_content):
    # Copy the original message content for this run
    modified_content = message_content.value[:]
    annotations = message_content.annotations if hasattr(message_content, 'annotations') else []
    citations = []

    for index, annotation in enumerate(annotations):
        # Replace the text with a footnote marker
        modified_content = modified_content.replace(annotation.text, f' [{index}]')

        # Process file citations and paths
        if (file_citation := getattr(annotation, 'file_citation', None)):
            cited_file = client.files.retrieve(file_citation.file_id)
            citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
        elif (file_path := getattr(annotation, 'file_path', None)):
            cited_file = client.files.retrieve(file_path.file_id)
            citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
            # Placeholder for actual file download link or method

    # Append gathered citations, ensuring they start on a new line
    if citations:
        modified_content += '\n\n' + '\n'.join(citations)  # Ensure two newlines before starting the citations

    # Wrap the modified content, including citations, with 50 character line width
    wrapped_content = textwrap.fill(modified_content, width=50)

    return wrapped_content



In [ ]:
original_message_content=message_info.content[0].text

# Use the function to process and wrap your content, adjust function parameters as needed
wrapped_and_processed_content = process_citations(client, original_message_content)

# Print the wrapped and processed content with correct newline before citations
print(wrapped_and_processed_content)